In [22]:
import pandas as pd
# Load datasets
train_df = pd.read_csv('E:/project 5 depresion/train.csv')
test_df = pd.read_csv('E:/project 5 depresion/test.csv')
train_df

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140695,140695,Vidya,Female,18.0,Ahmedabad,Working Professional,NaN,NaN,5.0,NaN,NaN,4.0,5-6 hours,Unhealthy,Class 12,No,2.0,4.0,Yes,1
140696,140696,Lata,Female,41.0,Hyderabad,Working Professional,Content Writer,NaN,5.0,NaN,NaN,4.0,7-8 hours,Moderate,B.Tech,Yes,6.0,5.0,Yes,0
140697,140697,Aanchal,Female,24.0,Kolkata,Working Professional,Marketing Manager,NaN,3.0,NaN,NaN,1.0,More than 8 hours,Moderate,B.Com,No,4.0,4.0,No,0
140698,140698,Prachi,Female,49.0,Srinagar,Working Professional,Plumber,NaN,5.0,NaN,NaN,2.0,5-6 hours,Moderate,ME,Yes,10.0,1.0,No,0


In [ ]:
import pandas as pd
import torch
from torch import nn, optim
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load training dataset
df = pd.read_csv('E:/project 5 depresion/train.csv')

# Clean column names
df.columns = df.columns.str.strip()

# Drop irrelevant columns if any
columns_to_drop = ['id', 'Name', 'City','Profession','Degree'] 
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Define target column
target_col = 'Depression'

# Separate feature types
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if target_col in numerical_cols:
    numerical_cols.remove(target_col)

# Handle missing values
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')

df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])
df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=categorical_cols)

# Normalize numeric features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Split features and target
X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

# Ensure X is fully numeric and compatible with PyTorch
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)
X = X.astype(float)

# Train-validation split (use stratify=y for balanced split)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convert to tensors 
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

# Define model architecture
input_size = X.shape[1]
hidden_size = [128, 64, 32, 16, 8, 4, 2]
output_size = 2  # Binary classification (0 or 1)

class DNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size[0]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[2], hidden_size[3]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[3], hidden_size[4]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[4], hidden_size[5]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[5], hidden_size[6]),
            nn.LeakyReLU(),
            nn.Linear(hidden_size[6], output_size)
        )

    def forward(self, x):
        return self.layers(x)

# Initialize model, loss function, and optimizer
model = DNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with validation metrics
num_epochs = 1000
for epoch in range(1, num_epochs + 1):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)

            val_preds = torch.argmax(val_outputs, dim=1).cpu().numpy()
            y_val_np = y_val_tensor.cpu().numpy()

            val_acc = accuracy_score(y_val_np, val_preds)
            val_prec = precision_score(y_val_np, val_preds, zero_division=0)
            val_rec = recall_score(y_val_np, val_preds, zero_division=0)
            val_f1 = f1_score(y_val_np, val_preds, zero_division=0)

        print(f"Epoch {epoch}/{num_epochs} - "
              f"Train Loss: {loss.item():.4f} - "
              f"Val Loss: {val_loss.item():.4f} - "
              f"Val Acc: {val_acc:.4f} - "
              f"Val Precision: {val_prec:.4f} - "
              f"Val Recall: {val_rec:.4f} - "
              f"Val F1: {val_f1:.4f}")

# Load test dataset
test_df = pd.read_csv('E:/project 5 depresion/test.csv')

# Clean test dataset same way
test_df.columns = test_df.columns.str.strip()
test_df = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])

# Apply imputers and scaler on test set
test_df[categorical_cols] = cat_imputer.transform(test_df[categorical_cols])
test_df[numerical_cols] = num_imputer.transform(test_df[numerical_cols])
test_df = pd.get_dummies(test_df, columns=categorical_cols)

# Align test set columns with train columns
missing_cols = set(X.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0
test_df = test_df[X.columns]  

# Scale numerical columns
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

# Ensure all test data is numeric
test_df = test_df.apply(pd.to_numeric, errors='coerce')
test_df = test_df.fillna(0).astype(float)

# Convert to tensor
X_test_tensor = torch.tensor(test_df.values, dtype=torch.float32)


# Evaluate on test set (no labels assumed)
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_preds = torch.argmax(test_outputs, dim=1)

print("Test predictions (class indices):")
print(test_preds.numpy())


# Save to CSV
submission = pd.DataFrame({'prediction': test_preds.numpy()})
submission.to_csv('test_predictions.csv', index=False)

# Save model state_dict for deployment
torch.save(model.state_dict(), 'dnn_depression_model.pth')


Epoch 100/1000 - Train Loss: 0.2424 - Val Loss: 0.2395 - Val Acc: 0.9068 - Val Precision: 0.9178 - Val Recall: 0.5347 - Val F1: 0.6757
Epoch 200/1000 - Train Loss: 0.1551 - Val Loss: 0.1577 - Val Acc: 0.9375 - Val Precision: 0.8417 - Val Recall: 0.8077 - Val F1: 0.8244
Epoch 300/1000 - Train Loss: 0.1530 - Val Loss: 0.1560 - Val Acc: 0.9376 - Val Precision: 0.8390 - Val Recall: 0.8124 - Val F1: 0.8255
Epoch 400/1000 - Train Loss: 0.1522 - Val Loss: 0.1556 - Val Acc: 0.9378 - Val Precision: 0.8400 - Val Recall: 0.8122 - Val F1: 0.8259
Epoch 500/1000 - Train Loss: 0.1513 - Val Loss: 0.1554 - Val Acc: 0.9378 - Val Precision: 0.8403 - Val Recall: 0.8119 - Val F1: 0.8258
Epoch 600/1000 - Train Loss: 0.1504 - Val Loss: 0.1556 - Val Acc: 0.9372 - Val Precision: 0.8410 - Val Recall: 0.8070 - Val F1: 0.8236
Epoch 700/1000 - Train Loss: 0.1495 - Val Loss: 0.1558 - Val Acc: 0.9373 - Val Precision: 0.8441 - Val Recall: 0.8036 - Val F1: 0.8234
Epoch 800/1000 - Train Loss: 0.1486 - Val Loss: 0.1561 

In [2]:
# Save to CSV
submission = pd.DataFrame({'prediction': test_preds.numpy()})
submission.to_csv('test_predictions.csv', index=False)


In [3]:
test_preds

tensor([0, 0, 0,  ..., 0, 1, 0])

In [4]:
import pickle

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open("imputers.pkl", "wb") as f:
    pickle.dump((cat_imputer, num_imputer), f)

with open("reference_columns.pkl", "wb") as f:
    pickle.dump(X.columns.tolist(), f)  # Save column order used during training
